LGBM with random split for early stopping

- the lightGBM model hav ebeen replaced with xgboost and the code has been updated accordingly.
- i am also firring VotingClassifiers of RandomForests and ensembling the results of the XGBs with the RFs.
- some additional features have been added.
- some features which were previously dropped have been retained.
- some of the code has been reorganized.
- Rather than splitting the data once and using the validation data for the LGBM earyl stopping, I split the dat aduring the training so the entire training set can be trained on.

in this kernel :

1. training on the heads of households only
2. balance class frequencies
3. this kernel uses macro F1 score to early stopping in training
4. OHE if reversed into label encoding, as it is easier to digest for a tree model.
5. idhogar is NOT used in training.
6. There are aggregations done within households and new features are hand-crafted
7. A voting classifier is used to average over several LGBM models


In [5]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import f1_score
import joblib
from joblib import Parallel, delayed
from sklearn.base import clone
from sklearn.ensemble import VotingClassifier, ExtraTreesClassifier, RandomForestClassifier
from sklearn. utils import class_weight
#if 'balanced', class weights will be given by n_smaples / (n_classes * np,bincount(y))


import warnings
warnings.filterwarnings('ignore')

In [11]:
from sklearn.preprocessing import LabelEncoder

#this only transofrms the idhogar filed, the other things this functino used to do are done elsewhere
def encode_data(df) :
  df['idhogar'] = LabelEncoder().fit_transofrm(df['idhogar'])


#plot feature importance for sklearn decision trees
def feature_importance(forest, x_train, display_results = True) :
  ranked_list = []
  zero_features = []

  importances = forest.feature_importances_

  indices = np.argsort(importances)[::-1]

  if display_results :
    #print the feature ranking
    print('FEature ranking')

    for f in range(x_train.shape[1]) :
      if display_results :
        print('%d. feature %d (%f)' % (f+1, indices[f],importances[indices[f]] + " _ " + x_train.columns[indices[f]]))
        ranked_list.append( x_train.columns[indices[f]] )

        if importances[indices[f]] == 0.0 :
          zero_features.append(x_train.columns[indices[f]])

  return ranked_list, zero_features
  

In [12]:
def do_features(df) :
  feats_div = [('children_fraction', 'r4t1', 'r4t3'), 
                 ('working_man_fraction', 'r4h2', 'r4t3'),
                 ('all_man_fraction', 'r4h3', 'r4t3'),
                 ('human_density', 'tamviv', 'rooms'),
                 ('human_bed_density', 'tamviv', 'bedrooms'),
                 ('rent_per_person', 'v2a1', 'r4t3'),
                 ('rent_per_room', 'v2a1', 'rooms'),
                 ('mobile_density', 'qmobilephone', 'r4t3'),
                 ('tablet_density', 'v18q1', 'r4t3'),
                 ('mobile_adult_density', 'qmobilephone', 'r4t2'),
                 ('tablet_adult_density', 'v18q1', 'r4t2'),
                ]
  feats_sub = [('people_not_living', 'tamhog', 'tamviv'),
                 ('people_weird_stat', 'tamhog', 'r4t3')]

  for f_new, f1, f2 in feats_div :
    df['fe_' + f_new] = (df[f1] / df[f2]).astype(np.float32)

  for f_new, f1, f2 in feats_sub :
    df['fe_' + f_new] = (df[f1] / df[f2]).astype(np.float32)


    #aggregation rules over hosehold
    aggs_num = {'age' : ['min','max','mean'],
                'escolari' : ['min','max','mean']}

    aggs_cat = {'dis' : ['mean']}
  
  for s_ in ['estadicivil','parentesco','instlevel'] :
    for f_ in [f_ for f_ in df.columns if f_.startswith(s_)] :
      aggs_cat[f_] = ['mean','count']

  
  #aggregation over household
  for name_, df_ in [('18', df.query('age >= 18'))] :
    df_agg = df_.groupby('idhogar').agg({**aggs_num, **aggs_cat}).astype(np.float32)

    df_agg.columns = pd.Index(['agg' + name_ + '_' + e[0] + '_' + e[1].upper() for e in df_agg.columns.tolist()])

    df= df.join(df_agg, how='left', on='idhogar')
    del df_agg

  #drop id's
  df.drop(['Id'], axis=1, inplace=True)

  return df

In [13]:
def conver_OHE2Le(df) :
  tmp_df = df.copy(deep=True)
  for s_ in ['pared','piso','techo','abastagua','sanitario','energcocinar','elimbase','epared','etecho','eviv','estadicivil','parentesco','instlevle','lugar','tipovivi','manual_elec'] :
    if'manual_' not in s_:
      cols_s_ = [f_ for f_ in df.columns if f_.startswith(s_)]
    elif 'elec' in s_ :
      cols_s_ = ['public','planpri','noelec','coopele']
      sum_ohe = tmp_df[cols_s_].sum(axis=1).unique()
      #deal with those ohe, where there is a sum over columns ==0

    
    if 0 in sum_ohe :
      print('the ohe in {} is incomplete. a new column will be added before label encoding'.format(s_))

      #dummy column name to be added
      col_dummy = s_ + '_dummy'
      #add the column to the dataframe
      tmp_df[col_dummy] = (tmp_df[cols_s_].sum(axis=1) ==0).astype(np.int8)
      #add the name to the list of columns to be label-encoded
      cols_s_.append(col_dummy)

      # proof-check, that now the category is complete
      sum_ohe = tmp_df[cols_s_].sum(axis=1).unique()
      if -0 in sum_ohe :
        print('The category completeion did not work')

    tmp_cat = tmp_df[cols_s_].idxmax(axis=1)
    tmp_df[s_ + '_LE'] = LabelEncoder().fit_transform(tmp_cat).astype(np.int16)
    if 'parentesco1' in cols_s_ :
      cols_s_.remove('parentesco1')
      tmp_df.drop(cols_s_, axis=1, inplace=True)
  return tmp_df

Read in the dat aand clean it up

In [ ]:
train=pd.read_csv('../input/train.csv')
test=pd.read_csv('../input/test.csv')

test_ids = test.Id

In [ ]:
def process_df(df_):
  #encode the idhogar
  encode_data(df_)

  #create aggregate features
  return do_features(df_)

train=process_df(train)
test=process_df(test)

clean up some missing data and convert objects to numeric.

In [14]:
#some dependencies are na, fill those with the square root of the square


train['dependency'] =np.sqrt(train['SQBdependency'])
test['dependency'] =np.sqrt(test['SQBdependency'])

#fill 'no's for education with 0s

train.loc[train['edjefa'] == 'no', 'edjefa'] =0
train.loc[train['edjefe'] == 'no', 'edjefe'] =0
test.loc[test['edjefa'] == 'no', 'edjefa'] =0
test.loc[test['edjefa'] == 'no', 'edjefa'] =0

#if eduatino is 'yes' and person is head of households, fill with escolari

train.loc[(train['edjefa'] == 'yes') & (train['parnetesci1'] ==1), 'edjefa'] = train.loc[(train['edjefa']=='yes') & (train.loc['parentesco1'] ==1), 'escolari']
train.loc[(train['edjefe'] == 'yes') & (train['parnetesci1'] ==1), 'edjefe'] = train.loc[(train['edjefe']=='yes') & (train.loc['parentesco1'] ==1), 'escolari']
test.loc[(test['edjefa'] == 'yes') & (test['parnetesci1'] ==1), 'edjefa'] = test.loc[(test['edjefa']=='yes') & (test.loc['parentesco1'] ==1), 'escolari']
test.loc[(test['edjefe'] == 'yes') & (test['parnetesci1'] ==1), 'edjefe'] = test.loc[(test['edjefe']=='yes') & (test.loc['parentesco1'] ==1), 'escolari']


#convert to int for our models
train['edjefe'] = train['edjefe'].astype('int')
train['edjefa'] = train['edjefa'].astype('int')
test['edjefe'] = test['edjefe'].astype('int')
test['edjefa'] = test['edjefa'].astype('int')

#create feature with max education of either head of household
train['edjef'] = np.max(train[['edjefe','edjefa']], axis=1)
test['edjef'] = np.max(test[['edjefe','edjefa']], axis=1)

#fill some nas
train['v2a1'] = train['v2a1'].fillna(0)
test['v2a1'] =test['v2a1'].fillna(0)

train['v18q1'] = train['v18q1'].fillna(0)
test['v18q1'] =test['v18q1'].fillna(0)

train['rez_esc'] = train['rez_esc'].fillna(0)
test['rez_esc'] = test['rez_esc'].fillna(0)

train.loc[train.meaneduc.isnul(), 'meaneduc'] =0
test.loc[test.meaneduc.isnul(), 'meaneduc'] =0

train.loc[train['SQBmeaned'.isnull(), 'SQBmeaned']] = 0
test.loc[test['SQBmeaned'.isnull(), 'SQBmeaned']] = 0

#fix some inconssistencies int he data
train.loc[(train.v14a ==  1) & (train.sanitario1 ==  1) & (train.abastaguano == 0), "v14a"] = 0
train.loc[(train.v14a ==  1) & (train.sanitario1 ==  1) & (train.abastaguano == 0), "sanitario1"] = 0

test.loc[(test.v14a ==  1) & (test.sanitario1 ==  1) & (test.abastaguano == 0), "v14a"] = 0
test.loc[(test.v14a ==  1) & (test.sanitario1 ==  1) & (test.abastaguano == 0), "sanitario1"] = 0

NameError: ignored

In [ ]:
def train_test_apply_func(train_, test_, func_) :
  test_['Target'] = 0
  xx = pd.concat([train_, test_])

  xx_func = func_(xx)
  train_ = xx.func.iloc[: train_.shape[0], :]
  test_ = xx.func.iloc[: train_.shape[0], :].drop('Target', axis=1)

  del xx, xx_func
  return train_, test_

In [ ]:
#convert the one hot fields into label encoded

train, test = train_test_apply_func(train, test, convert_OHE2LE)

Geo aggregates

In [ ]:
cols_2_ohe = ['eviv_Le', 'etecho_Le','epared_LE','elimbasu_LE', 'energcocinar_Le','sanirario_LE','manual_elec_Le', 'pared_LE']
cols_nums = ['age','meaneduc','dependency','hogar_nin','hogar_adul' 'hogar_mayor','hogar_total','bedrooms','overcrowding']

def convert_geo2aggs(df_) :
  tmp_df = pd.concat([df_[(['lugar_Le','idhogar'] + cols_nums)], pd.get_dummies(df_[cols_2_ohe],columns=cols_2_ohe)], axis=1)

  geo_agg = tmp_df.groupby(['lugar_LE','idhogar']).mean().groupby('lugar_LE').mean().astype(np.float32)
  get_agg.columns = pd.Index(['geo_' + e for e in geo.agg.columns.tolist()])

  del tmp_df
  return df_.join(geo_agg, how='left', on='lugar_LE')

#add some aggregates by geography
train, test = train_test_apply_func(train, test, convert_geo2aggs)


In [ ]:
#add the number of people over 18 in each household

train['num_over_18'] = 0
train['num_over_18'] = train[train.age >= 18].groupby('idhogar').transform("count")
train['num_over_18'] = train.groupby("idhogar")["num_over_18"].transform("max")
train['num_over_18'] = train['num_over_18'].fillna(0)

test['num_over_18'] = 0
test['num_over_18'] = test[test.age >= 18].groupby('idhogar').transform("count")
test['num_over_18'] = test.groupby("idhogar")["num_over_18"].transform("max")
test['num_over_18'] = test['num_over_18'].fillna(0)

#add some extra features, these were taken from another kernel

def exract_feature(df) :
  df['bedrooms_to_rooms'] = df['bedroom'] / df['rooms']
  ...

In [ ]:
#drop duplicated columns
needless_cols = ['r4t3','tamhog','tamviv','hhsize','v18q','v14a','agesq','mobilephone','female']

instlevel_cols = [s for s in train.columns.tolist() if 'instlevel' in s]

needless_cols.extend(instlevel_cols)

train=train.drop(needless_cols, axis=1)
test=test.drop(needless_cols, axis=1)

##Split data

we split the data by household to avoid leakage, since rows belonging to the same houshold usually have the same target. SInce we filter the data to only include heads of household this isn't technically necessary, but it provedes an easy way to use the netire training data sey if we want to do that.

In [ ]:
def split_data(train,y,sample_weigh=None, households=None,test_percentage=0.2,seed=None) :
  train2=train.copy()

  #pick some random households to use for the test data
  cv_hhs = np.random.choice(households, size=int(len(households) * test_percentage), replace=False)

  #select households which are in the random selection
  cv_idx = np.isin(households, cv_hhs)
  x_test = train2[cv_idx]
  y_test = y[cv_idx]

  x_train=train2[~cv_idx]
  y_train=y[~cv_idx]

  if sample_weight is not None :
    y_train_weights = sample_weight[~cv_idx]
    return x_train, y_train, x_test, y_test, y_train_weights

  
  return x_train, y_train,x_test,y_test


In [ ]:
x = train.query('parnetesco1==1')
# x=train.copy()


#pull out and drop the target variable

y=x['Target']-1
x=x.drop(['Target'],axis=1)

np.random.seed(seed=None)

train2=x.copy()

train_hhs=train2.idhogar

households = train2.idhogar.unique()

cv_hhs = np.random.choice(households, size=int(len(households) * 0.15), replace=False)

cv_idx = np.isin(train2.idhogar, cv_hhs)


x_test = train2[cv_idx]
y_test = y[cv_idx]

x_train = train2[~cv_idx]
y_train = y[~cv_idx]

#train on entire dataset
x_train = train2
y_train = y

train_households=  x_train.idhogar

In [ ]:
#figure out the class weights for training with unbalanced classes
y_train_weights = class_weight.compute_sample_weight('balanced',y_train, indices=None)

In [ ]:
#drop some feature which aren't used by the LGBM or have very low importance
extra_drop_features = [
 'agg18_estadocivil1_MEAN',
 'agg18_estadocivil6_COUNT',
 'agg18_estadocivil7_COUNT',
 'agg18_parentesco10_COUNT',
 'agg18_parentesco11_COUNT',
 'agg18_parentesco12_COUNT',
 'agg18_parentesco1_COUNT',
 'agg18_parentesco2_COUNT',
 'agg18_parentesco3_COUNT',
 'agg18_parentesco4_COUNT',
 'agg18_parentesco5_COUNT',
 'agg18_parentesco6_COUNT',
 'agg18_parentesco7_COUNT',
 'agg18_parentesco8_COUNT',
 'agg18_parentesco9_COUNT',
 'geo_elimbasu_LE_4',
 'geo_energcocinar_LE_1',
 'geo_energcocinar_LE_2',
 'geo_epared_LE_0',
 'geo_hogar_mayor',
 'geo_manual_elec_LE_2',
 'geo_pared_LE_3',
 'geo_pared_LE_4',
 'geo_pared_LE_5',
 'geo_pared_LE_6',
 'num_over_18',
 'parentesco_LE',
 'rez_esc']

In [ ]:
xgb_drop_cols = extra_drop_features + ['idhogar','parentesco1']

##Fit a voting classifier

Define a derived VotingClassifier class to be able to pass fit_params for early stopping. Vote based on LGBM models with early stopping based on macro f1 and decaying learning rate.

In [ ]:
#if you need to set optimal paramters, use automl like lazypredict.

opt_parameters1 = {'max_depth' : 35,
                  'eta' : 0.1,
                  'silent' : 0,
                  'objective' : 'multi:softmax',
                  'min_child_weight' : 1,
                  'num_class' :'4',
                  'gamma' : 2, 'colsample_bylevel' : 0.9, 'subsample' :0.84, 'colsample_bytree' : 0.88, 'reg_labda':0.4}

opt_parameters2 = {'max_depth' : 35,
                  'eta' : 0.1,
                  'silent' : 0,
                  'objective' : 'multi:softmax',
                  'min_child_weight' : 1,
                  'num_class' :'4',
                  'gamma' : 2, 'colsample_bylevel' : 0.9, 'subsample' :0.84, 'colsample_bytree' : 0.88, 'reg_labda':0.4}


def evaluate_macroF1_lgb(predictions, truth) :
  pred_labels = predictions.argmax(axis=1)
  truth=truth.get_label()
  f1 = f1_score(truth, pred_labels, average='macro')
  return ('macroF1',1-f1)

fit_params = {'early_stopping_rounds' : 500, 'eval_metric' : evaluate_macroF1_lgb,
              'eval_set' : [(x_train,y_train), (x_test,y_test)],
              'verbose':500}

def learning_rate_power_0997(current_iter) :
  base_learning_rate = 0.1
  min_sampling_rate = 0.02
  lr = base_learning_rate * np.power(.995, current_iter)
  return max(lr, min_sampling_rate)

fit_params['verbose'] = 50

In [ ]:
np.random.seed(100)

def _parallel_fit_estimator(estimator1, x, y, sample_weight = None, threshold=True, **fit_params) :
  estimator = clone(estimator1)

  #randomly split the data so we have a test set for early stopping
  if sample_weight is not None :
    x_train,y_train,x_test,y_test,y_train_weight = split_data(x,y,sample_weight, households= train_households)

  else :
    x_train,y_train,x_test,y_test = split_data(x,y,None,households=train_households)

    #update the fit params with our new split
    fit_params['eval_set'] = [(x_test,y_test)]

    #fir the estimator
  if sample_weight is not None :
    if isinstance(estimator1, ExtraTreesClassifier) or isinstance(estimator1, RandomForestClassifier) :
      estimator.fit(x_train,y_train)
    else :
       _ = estimator.fit(x_train, y_train, sample_weight=y_train_weight, **fit_params)

  else :
    if isinstance(estimator1, ExtraTreesClassifier) or isinstance(estimator1, RandomForestClassifier) :
      estimator.fit(x_train,y_train)
    else :
      _ = estimator.fit(x_train,y_train, **fit_params)
    
  if not isinstance(estimator1, ExtraTreesClassifier) and not isinstance(estimator1, RandomForestClassifier) and not isinstance(estimator1, xgb.XGBClassiifer) :
    best_cv_round = np.argma(estimator.evals_result_['validation_0']['mlogloss'])
    best_cv = np.max(estimator.evals_result_['validation_0']['mlogloss'])
    best_train = estimator.evals_result_['train']['macroF1']['best_cv_round']
  else :
    best_train = f1_score(y_train, estimator.predict(x_train), average='macro')
    best_cv = f1_score(y_test, estimator.predict(x_test), average='macro')
    print('train f1 :', best_train)
    print('test f1 :', best_cv)

  #reject some estimators based on their performance on train and test sets
  if threshold :
    #if the vlaid score is very high, we'll allow a little more leeway with the train scroes
    if ((best_cv > 0.37) & (best_train > 0.75)) or ((best_cv>0.44) & (best_train > 0.65)) :
      return estimator

    #else recurse until we get a better one
    else :
      print('unacceptable. try again')
      return _parallel_fit_estimator(estimaotr1, x,y,sample_weight=sample_wiegh, **fit_params)
  else :
    return estimator



make votingclassifier with LGBM

In [ ]:
class VotingClassifierLGBM(VotingClassifier) :
  '''this implements the fit method of the VotingClassifier propagating fit_params'''
  def fit(self, x,y,sample_weight=None, threshold=True, **fit_params) :
    if isinstance(y,np.ndarray) and len(y.shape) > 1 and y.shape[1] > 1 :
      raise NotImplementedError('Multilabel and multi-output' 'classification is not supported')
    
    if self.voting not in ('soft','hard') :
      raise ValueError("voting must be 'soft' or 'hard' ; got(voting=%r)" % self.voting)
    
    if self.estimators is None or len(self.estimators) == 0 :
      raise AttributeError("Invalid estimators attribute, 'estimators'")

    names,clfs = zip(*self.estimators)
    self._validate_names(names)

    n_isnone = np.sum([clf is None for _, clf in self.estimators])
    if n_isnone == len(self.estimators) :
      raise ValueError("all estimators are none. at least one is"'required to be a classifier')
    
    self.le_ = LabelEncoder().fit(y)
    self.classes_ = self.le_.classes_
    self.estimators_ = []

    transformed_y = self.le_.transform(y)

    self.estimators_ = Parallel(n_job=self.n_jobs)(delayed(_parallel_fit_estimator)(clone(clf),x,transformed_y,sample_weight, threshold=threshold, **fit_params)
    for clf in clfs if clf is not None)

    return self

In [ ]:
clfs = []
for i in range(15) :
  clf = xgb.XGBClassifer(random_state=42, n_estimators=300, learning_rate =0.15, n_jobs=-1, **opt_parameters)

  clfs.append(('xgb{}'.format(i), clf))

vc = VotingClassifierLGBM(clfs, voting='soft')
del(clfs)

#train the final model with leaning rate decay

_ = vc.fit(x_train.drop(xgb_drop_cols, axis=1), y_train, sample_weight=y_train_weights, threshold=False, **fit_params)

clf_final = vc.estimators_[0]

In [ ]:
#params 4 - 400 early stop and 15 est, l1 used features-weighted

global_score = f1_score(y_test, clf_final.predict(x_test.drop(xgb_drop_cols, axis=1)), average='macro')
vc.voting='soft'
global_score_soft = f1_score(y_test, vc.predict(x_test.drop(xgb_drop_cols, axis=1)), average='macro')

vc.voting='hard'
global_score_hard = f1_score(y_test, vc.predict(x_test.drop(xgb_drop_cols, axis=1)), average='macro')

print('validation score of a single LGBM classiifer : {:.4f}'.format(global_score))

print('validation score of a votingclassifier on 3 lgbms with soft voring strategy : {:.4f}'.format(global_score_soft))

print('validation score of a votingclassifier on 3 lgbms with hard voring strategy : {:.4f}'.format(global_score_hard))



In [ ]:
#see which features are not used by ANY models
useless_features = []
drop_features = set()
counter=0

for est in vc.estimators_ :
  ranked_features, unused_features = feature_importance(est, x_train.drop(xgb_drop_cols, axis=1), display_results = False)
  useless_features.append(unused_features)
  if counter==0 :
    drop_features = set(unused_features)
  else :
    drop_features = drop_features.intersection(set(unused_features))
    counter+=1

drop_features


In [ ]:
ranked_features = feature_importance(clf_final, x_train.drop(xgb_drop_cols, axis=1))

RF

In [ ]:
#result cols from ranked_Features
et_drop_cols = ['agg18_age_MAX', 'agg18_age_MEAN', 'agg18_age_MIN', 'agg18_dis_MEAN',
       'agg18_escolari_MAX', 'agg18_escolari_MEAN', 'agg18_escolari_MIN',
       'agg18_estadocivil1_COUNT', 'agg18_estadocivil1_MEAN',
       'agg18_estadocivil2_COUNT', 'agg18_estadocivil2_MEAN',
       'agg18_estadocivil3_COUNT', 'agg18_estadocivil3_MEAN',
       'agg18_estadocivil4_COUNT', 'agg18_estadocivil4_MEAN',
       'agg18_estadocivil5_COUNT', 'agg18_estadocivil5_MEAN',
       'agg18_estadocivil6_COUNT', 'agg18_estadocivil6_MEAN',
       'agg18_estadocivil7_COUNT', 'agg18_estadocivil7_MEAN',
       'agg18_parentesco10_COUNT', 'agg18_parentesco10_MEAN',
       'agg18_parentesco11_COUNT', 'agg18_parentesco11_MEAN',
       'agg18_parentesco12_COUNT', 'agg18_parentesco12_MEAN',
       'agg18_parentesco1_COUNT', 'agg18_parentesco1_MEAN',
       'agg18_parentesco2_COUNT', 'agg18_parentesco2_MEAN',
       'agg18_parentesco3_COUNT', 'agg18_parentesco3_MEAN',
       'agg18_parentesco4_COUNT', 'agg18_parentesco4_MEAN',
       'agg18_parentesco5_COUNT', 'agg18_parentesco5_MEAN',
       'agg18_parentesco6_COUNT', 'agg18_parentesco6_MEAN',
       'agg18_parentesco7_COUNT', 'agg18_parentesco7_MEAN',
       'agg18_parentesco8_COUNT', 'agg18_parentesco8_MEAN',
       'agg18_parentesco9_COUNT', 'agg18_parentesco9_MEAN']

et_drop_cols.extend(['idhogar','parentesco1','fe_rent_per_person','fe_rent_per_room','fe_tablet_adult_density','fe_tablet_density'])

In [ ]:
#so the same thing for some extra trees classifiers

ets = []

for i in range(10) :
  rf = RandomForestClassifier(max_depth=None, random_state=42, n_jobs=-1, n_estimators=700, min_impurity)decrease=1e-3, min_samples_leaf = 2, verbose=0, class_weight='balanced')
  ets.append('rf{}'.format(i),rf)

vc2 = VotingClassifier(ets, voing='soft')
_ = vc2.fit(x_train.drop(et_drop_cols, axis=1), y_train, threshold=False)

In [ ]:
# w/ threshold, extra drop cols
vc2.voting = 'soft'

global_rf_score_soft= f1_score(y_test, vc2.predict(x_test.drop(et_drop_cols, axis=1)), average='macro')

vc2.votinh='hard'
global_rf_score_hard= f1_score(y_test, vc2.predict(x_test.drop(et_drop_cols, axis=1)), average='macro')

print('validation score of a votingclassifier on 3 lgbms with soft voting strategy : {:.4f}'.format(global_rf_score_soft))
print('validation score of a votingclassifier on 3 lgbms with hard voting strategy : {:.4f}'.format(global_rf_score_hard))

In [ ]:
#see which features are not used by any models


useless_features = []
drop_features = set()
counter = 0

for est in vc2.estimators_ :
  ranked_features, unused_features = feature_importance(est, x_train.drop(et_drop_cols, axis=1), display_results = False)
  useless_features.append(unused_features)

  if counter == 0 :
    drop_features = set(unused_features)
  else :
    drop_features = drop_features.intersection(set(unused_features))
    counter +=1

drop_features_vc2

In [ ]:
def combine_voters(data, weights=[0.5,0.5]) :
  #do soft voting with both classifiers
  vc.voting = 'soft'
  vc1_probs = vc.predict_proba(data.drop(xgb_drop_dols, axis=1))
  vc2.voting='soft'
  vc2_probs = vc2.predict_proba(data.drop(xgb_drop_cols, axis=1))

  final_vote = (vc1_probs * weights[0] + (vc2_probs * weights[1]))
  predictions = np.argmax(final_vote, axis=1)

  return predictions

In [ ]:
combo_preds = combine_voters(x_test, weights=[0.5,0.5])

global_combo_score_soft = f1_score(y_test, combo_preds, average='macro')
global_combo_socre_soft

In [ ]:
#if change weights :
combo_preds = combine_voters(x_test, weights=[0.5,0.5])

global_combo_score_soft = f1_score(y_test, combo_preds, average='macro')
global_combo_socre_soft

#Prepare submission

In [ ]:
y_sub = pd.DataFrame()
y_sub['Id'] = test.ids

In [ ]:
vc.voting='soft'

y_sub_lgb = y_sum.copy(deep=True)
y_sub_lgb['Target'] = vc.predict(test.drop(xgb_drop_cols, axis=1)) +1

vc2.voting='soft'
y_sub_lgb = y_sub.copy(deep=True)
y_sub_rf['Target'] = vc2.predict(test.drop(et_drop_cols, axis=1))+1

y_sub_ens = y_sub.copy(deep=True)
y_sub_ens['Target'] = combine_voters(test)+1

In [ ]:
from datetime import datetime
now = datetime.now()

sub_file_lgb = 'submission_soft_xgb_{:.4f}_{}.csv'.format(gloabal_score_soft, str(now.strftime('%Y-%M-%d-%h-%m')))

sub_file_rf = 'submission_soft_rf_{:4.f}_{}.csv'.format(global_rf_score_soft, str(now.strftime('%y-%m-%d-%h-%m')))

sub_file_ens = 'submission_ens_{:.4f}_{}.csv'format(global_rf_score_soft, str(now.strftime('%y-%m-%d-%h-%m')))

y_sub_lgb.to_csv(sub_file_lgb, index=False)
y_sub_rf.to_csv(sub_file_lgb, index=False)
y_sub_ens.to_csv(sub_file_lgb, index=False)
